# Import

In [9]:
import timm
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Subset, random_split, Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt
from tqdm import tqdm

## Deform Images

In [3]:
def sin_distortion(x_length: int,
                   y_length: int,
                   A_nm: torch.Tensor) -> (torch.Tensor, torch.Tensor):
    """
    Sin distortion for creating deformation maps.

    Args:
    - x_length (int): Length of x-axis of image.
    - y_length (int): Length of y-axis of image.
    - A_nm (torch.Tensor): Square matrix of coefficients. Sets size of cut off.

    Returns:
    (torch.Tensor, torch.Tensor): Deformation maps for x and y coordinates.
    """
    if A_nm.shape[0] != A_nm.shape[1]:
        raise ValueError('A_nm must be square matrix.')

    A_nm = A_nm.float()

    # Create Coordinates
    x = torch.linspace(-1, 1, x_length, dtype=torch.float32)
    y = torch.linspace(-1, 1, y_length, dtype=torch.float32)
    X, Y = torch.meshgrid(x, y, indexing='ij')

    # Create Diffeo
    x_pert = torch.linspace(0, 1, x_length, dtype=torch.float32)
    y_pert = torch.linspace(0, 1, y_length, dtype=torch.float32)

    n = torch.arange(1, A_nm.shape[0] + 1, dtype=torch.float32)
    x_basis = torch.sin(torch.pi * torch.outer(n, x_pert)).T
    y_basis = torch.sin(torch.pi * torch.outer(n, y_pert))

    perturbation = torch.matmul(x_basis, torch.matmul(A_nm, y_basis))

    x_map = X + perturbation
    y_map = Y + perturbation

    return x_map, y_map

def apply_transformation(image_tensor,
                         A_nm: torch.Tensor,
                         interpolation_type='bilinear'):
    """
    Wrapper of `sin_distortion`. Gets torch.tensor and returns the distorted
    torch.tensor according to A_nm.

    Args:
        image_tensor (torch.Tensor): Inputted image.
        A_nm (torch.Tensor): Characterizes diffeo according to `sin_distortion`.
        interpolation_type (str): Interpolation method ('bilinear' or 'nearest').

    Returns:
        image_tensor_deformed (torch.Tensor): Diffeo applied to `image_tensor`.
    """
    # Create deformation map
    x_length, y_length = image_tensor.shape[1:3]
    x_map, y_map  = sin_distortion(x_length, y_length, A_nm)

    return apply_flowgrid(image_tensor, x_map, y_map, interpolation_type=interpolation_type)


def apply_flowgrid(image_tensor, x_map, y_map, interpolation_type='bilinear'):
    # Stack and unsqueeze to form grid
    grid = torch.stack((y_map, x_map), dim=-1).unsqueeze(0).to(image_tensor.device)

    # Apply grid sample
    image_tensor_deformed = torch.nn.functional.grid_sample(image_tensor.unsqueeze(0),
                                                            grid,
                                                            mode=interpolation_type,
                                                            align_corners=True)

    return image_tensor_deformed.squeeze(0)

In [4]:
def diffeo_dataset(tensor):
    A_nm = torch.tensor([[0.0, 0.14],
                         [-0.02, 0.01]])
    return apply_transformation(tensor, A_nm)

# Get ImageNet

In [11]:
import torch as t

root = '/imagenet/'

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),
])
preprocess_diffeo = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),
    diffeo_dataset,
])


split = 'train'
dataset_images_og = torchvision.datasets.ImageNet(root=root, 
                                                  split=split, 
                                                  transform=preprocess,)
dataset_images_diffeo = torchvision.datasets.ImageNet(root=root, 
                                                      split=split, 
                                                      transform=preprocess_diffeo,)

# Image Classification Model

In [121]:
def get_activation(model, input, layer_index: list):

  activation = {}
  def getActivation(name):
      # the hook signature
      def hook(model, input, output):
          activation[name] = output.detach()
      return hook

  handles = []
  def retrieve_layer_activation(model, input, layer_index):
    if len(input) == 3: input = input[None, :, :, :]

    layers = list(model.children())
    layers_flat = flatten(layers)

    for index in layer_index:
      handles.append(layers_flat[index - 1].register_forward_hook(getActivation(str(index))))

    with t.no_grad(): model(input)
    for handle in handles: handle.remove()

    return

  def flatten(array):
      result = []
      for element in array:
          if hasattr(element, "__iter__"):
              result.extend(flatten(element))
          else:
              result.append(element)
      return result

  retrieve_layer_activation(model, input, layer_index)
  return activation

class Paired(Dataset):
    def __init__(self, dataset1, dataset2):
        assert len(dataset1) == len(dataset2), "Datasets must have the same length"
        self.dataset1 = dataset1
        self.dataset2 = dataset2

    def __len__(self):
        return len(self.dataset1)

    def __getitem__(self, idx):
        img1, _ = self.dataset1[idx]  # Get image from dataset1
        img2, _ = self.dataset2[idx]  # Get label from dataset2
        return img1, img2

In [122]:
##########
# Config
##########

# Which pictures to select
id_start = 1300
id_finish = id_start + 1300

# Which model to select
model_name = 'ResNet18'
model = torchvision.models.resnet18(pretrained=True)
model = model.to(device)

# Which layer of the model to select
layer_id = 13


######################
# Get Activations!
######################
subset_og = Subset(dataset_images_og, indices=range(id_start, id_finish))
subset_diffeo = Subset(dataset_images_diffeo, indices=range(id_start, id_finish))
paired_dataset = Paired(subset_og, subset_diffeo)
dataloader = DataLoader(paired_dataset, num_workers=4, batch_size = 100)

activation_og_lst = []
activations_diffeo_lst = []
for og_batch, diffeo_batch in tqdm(dataloader):
    activation_og = get_activation(model, og_batch, [layer_id])[f'{layer_id}'].flatten(start_dim=1).to('cpu')
    activation_diffeo = get_activation(model, diffeo_batch, [layer_id])[f'{layer_id}'].flatten(start_dim=1).to('cpu')
    activation_og_lst.append(activation_og)
    activations_diffeo_lst.append(activation_diffeo)
    

100%|██████████| 13/13 [00:32<00:00,  2.49s/it]


In [124]:
final_dataset = TensorDataset(torch.cat(activation_og_lst).to('cpu'), torch.cat(activations_diffeo_lst).to('cpu'))

torch.save(final_dataset, f"{model_name}_layer{layer_id}_ImageNet1kTrain_GoldfishOnly.pt")